Decision Trees are a pretty rudimentary but okay method of determining something.
Unfortunately, decision trees are hard-coded (which goes against the whole point of ML)
and for complex data can be tedious to make.

What if we could generate a decision tree though? A simple heuristic we can do is to
greedily split the data in a way that maximizes the uniqueness of each half. Essentially,
what this means is that we want to partition the data so that the points in each partition
are of the same class. Then, if a new point is in that partition, then it's likely to be in
that class. In this simple model, we only split along a single feature, although there are
obvious shortcomings with that.

To quantify this, there is a statistic called the Gini impurity which represents what we're looking for.
So, we want to greedily pick splits that minimizes the gini impurity. Of course, greedily splitting will
not necessarily lead to a partition that has the optimial (aka minimal) gini impurity.

In [2]:
import numpy as np
import pandas as pd

In [3]:
def gini(values):
    counts = {}
    for x in values:
        if x not in counts:
            counts[x] = 1
        else:
            counts[x] += 1
    sum = 0
    for c in counts.values():
        f = c/len(values)
        sum += f*f
    return 1-sum

In [90]:
#overfitting parameters
impurity_threshold = 0.05
max_depth = 3

In [91]:
classes = pd.read_csv("../datasets/decisiontree/sample.csv")
        

In [92]:
classes

,gender,car_type,shirt_size,class
0,M,Family,Small,C0
1,M,Sports,Medium,C0
2,M,Sports,Medium,C0
3,M,Sports,Large,C0
4,M,Sports,Extra Large,C0
5,M,Sports,Extra Large,C0
6,F,Sports,Small,C0
7,F,Sports,Small,C0
8,F,Sports,Medium,C0
9,F,Luxury,Large,C0


In [93]:
len(classes.groupby('gender'))

2

In [94]:
for group in classes.groupby('gender'):
    print(type(group[1]))
    print(group)

<class 'pandas.core.frame.DataFrame'>
('F',    gender car_type shirt_size class
6       F   Sports      Small    C0
7       F   Sports      Small    C0
8       F   Sports     Medium    C0
9       F   Luxury      Large    C0
14      F   Luxury      Small    C1
15      F   Luxury      Small    C1
16      F   Luxury     Medium    C1
17      F   Luxury     Medium    C1
18      F   Luxury     Medium    C1
19      F   Luxury      Large    C1)
<class 'pandas.core.frame.DataFrame'>
('M',    gender car_type   shirt_size class
0       M   Family        Small    C0
1       M   Sports       Medium    C0
2       M   Sports       Medium    C0
3       M   Sports        Large    C0
4       M   Sports  Extra Large    C0
5       M   Sports  Extra Large    C0
10      M   Family        Large    C1
11      M   Family  Extra Large    C1
12      M   Family       Medium    C1
13      M   Luxury  Extra Large    C1)


In [95]:
for column in classes.columns:
    print(type(column))
    print(column)

<class 'str'>
gender
<class 'str'>
car_type
<class 'str'>
shirt_size
<class 'str'>
class


In [96]:
classes['class']

0     C0
1     C0
2     C0
3     C0
4     C0
5     C0
6     C0
7     C0
8     C0
9     C0
10    C1
11    C1
12    C1
13    C1
14    C1
15    C1
16    C1
17    C1
18    C1
19    C1
Name: class, dtype: object

In [97]:
type(classes['class'])

pandas.core.series.Series

In [98]:
classes['class'].data

In [104]:
#takes in pandas dataframe
def discrete_split(df):
    initial_gini = gini(df['class'])
    best_gini_improvement = impurity_threshold
    split = None
    best_column = None
    for column in df.columns:
        if column == 'class':
            continue
        gini_split = 0
        groups = df.groupby(column)
        for c,group in groups:
            gini_split += gini(group['class'])
        gini_split /= len(groups)
        improvement = initial_gini-gini_split
        if improvement > best_gini_improvement:
            split = groups
            best_gini_improvement = improvement
            best_column = column
    return (split, best_column)

In [105]:
def most_common(values):
    counts = {}
    for x in values:
        if x not in counts:
            counts[x] = 1
        else:
            counts[x] += 1
    return max(counts,key=lambda x: counts[x])

In [106]:
def discrete_decision_tree(df,depth=0):
    if depth >= max_depth:
        prediction = most_common(df['class'])
        return (lambda x: prediction)
    #print(discrete_split(df))
    split,column = discrete_split(df)
    if split is None:
        prediction = most_common(df['class'])
        return (lambda x: prediction)
    else:
        child_trees = {}
        for c, group in split:
            child_trees[c] = discrete_decision_tree(group,depth+1)
        def predict(series):
            #print(series)
            #print(column)
            assert column in series
            assert series[column] in child_trees
            return child_trees[series[column]](series)
        return predict

In [107]:
predict = discrete_decision_tree(classes)
test_predict_data = classes.drop('class',axis=1)
for i in range(0,len(test_predict_data)):
    print(predict(test_predict_data[i:i+1].squeeze()))

C0
C0
C0
C0
C0
C0
C0
C0
C0
C0
C1
C1
C1
C1
C1
C1
C1
C1
C1
C0
